In [1]:
# プログラムを実行前に pip コマンドで以下をインストール
# (google colab の場合は以下のコメントアウトを外す)
#!pip install mecab-python3
#!pip install unidic
#!python -m unidic download
#!pip install sklearn

In [1]:
import MeCab
import unidic

sample_txt = 'さいたま市は、埼玉県南部にある政令市です。'

# MeCab により sample_txt の形態素解析を実施
tagger = MeCab.Tagger()
result = tagger.parse(sample_txt)
print(result)

# 解析結果から、単語とその品詞を出力
for token in result.split('\n'):
    if token != 'EOS' and len(token) > 0:
        _word, _mecab_output = token.split('\t') # 解析結果をタブで分割 -> 分かち書きされた単語 と MeCab の出力
        _mecab_result = _mecab_output.split(',') # MeCab の出力をカンマで分割
        print(_word, _mecab_result[0])

さいたま	名詞,固有名詞,地名,一般,,,サイタマ,サイタマ,さいたま,サイタマ,さいたま,サイタマ,固,"","","","","","",地名,サイタマ,サイタマ,サイタマ,サイタマ,"1","","",3783977890619904,13766
市	名詞,普通名詞,一般,,,,シ,市,市,シ,市,シ,漢,"","","","","","B1S8SjS,B1SjS",体,シ,シ,シ,シ,"1","C3","",4105585041744384,14936
は	助詞,係助詞,,,,,ハ,は,は,ワ,は,ワ,和,"","","","","","",係助,ハ,ハ,ハ,ハ,"","動詞%F2@0,名詞%F1,形容詞%F2@-1","",8059703733133824,29321
、	補助記号,読点,,,,,,、,、,,、,,記号,"","","","","","",補助,,,,,"","","",6605693395456,24
埼玉	名詞,固有名詞,地名,一般,,,サイタマ,サイタマ,埼玉,サイタマ,埼玉,サイタマ,固,"","","","","","",地名,サイタマ,サイタマ,サイタマ,サイタマ,"1","","",3783977957728768,13766
県	名詞,普通名詞,一般,,,,ケン,県,県,ケン,県,ケン,漢,"ケ濁","基本形","","","","B1S6SjShS",体,ケン,ケン,ケン,ケン,"1","C3","",3094034344190464,11256
南部	名詞,普通名詞,一般,,,,ナンブ,南部,南部,ナンブ,南部,ナンブ,漢,"","","","","","",体,ナンブ,ナンブ,ナンブ,ナンブ,"1","C1","",7740570483171840,28160
に	助詞,格助詞,,,,,ニ,に,に,ニ,に,ニ,和,"","","","","","",格助,ニ,ニ,ニ,ニ,"","名詞%F1","",7745518285496832,28178
ある	動詞,非自立可能,,,五段-ラ行,連体形-一般,アル,有る,ある,アル,ある,アル,和,"","","","","","",用,アル,アル,アル,アル,"1","C3","",334260158472897,1216
政令	名詞,普通名詞,一般,,,,セイレイ,政令,政令,

In [2]:
# 入力文を分かち書き(単語ごとに)した結果を出力
sample_txt = 'さいたま市は、埼玉県南部にある政令市です。'
tagger = MeCab.Tagger('-Owakati')
result = tagger.parse(sample_txt)
print(result)
for _word in result.split(' '):
    print(_word)

さいたま 市 は 、 埼玉 県 南部 に ある 政令 市 です 。 

さいたま
市
は
、
埼玉
県
南部
に
ある
政令
市
です
。




In [13]:
# BOW 計算用
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def init_bow_vectorizer(_text_list):
    '''文書集合中の文書を分かち書きして出現する単語から CountVectorizer を準備'''
    tagger = MeCab.Tagger('-Owakati')
    _wakati_list = []
    for _text in _text_list:
        _wakati_list.append(tagger.parse(_text).strip())
    
    vectorizer = CountVectorizer(token_pattern=u'(?u)\\b\\w+\\b')
    _np_array = np.array(_wakati_list)
    vectorizer.fit(_np_array)
    
    return vectorizer

def get_bow_vector(vectorizer, _sentence):
    '''CountVectorizer により _sentence から Bow特徴量を作成'''
    tagger = MeCab.Tagger('-Owakati')
    return vectorizer.transform(np.array([tagger.parse(_sentence).strip()])).toarray()[0]

In [16]:
_answers = [{'ans_text': 'さいたま市は、埼玉県南部にある政令市です。', 'vector': []},
            {'ans_text': 'さいたま市は、埼玉県北部にある政令市です。', 'vector': []},
            {'ans_text': 'さいたま市は、埼玉県南部にある都市です。', 'vector': []},]

_answer_texts = []
for _answer in _answers:
    _answer_texts.append(_answer['ans_text'])
_bow_vectorizer = init_bow_vectorizer(_answer_texts)

print(_bow_vectorizer.vocabulary_)

for _answer in _answers:
    _answer['vector'] = get_bow_vector(_bow_vectorizer, _answer['ans_text'])
    print(_answer)

{'さいたま': 1, '市': 8, 'は': 4, '埼玉': 7, '県': 10, '南部': 6, 'に': 3, 'ある': 0, '政令': 9, 'です': 2, '北部': 5, '都市': 11}
{'ans_text': 'さいたま市は、埼玉県南部にある政令市です。', 'vector': array([1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 0], dtype=int64)}
{'ans_text': 'さいたま市は、埼玉県北部にある政令市です。', 'vector': array([1, 1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 0], dtype=int64)}
{'ans_text': 'さいたま市は、埼玉県南部にある都市です。', 'vector': array([1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1], dtype=int64)}


In [20]:
def cos_sim(x, y):
    return float(np.dot(x, y) / (np.sqrt(np.sum(x ** 2)) * np.sqrt(np.sum(y ** 2))))

for _answer_1 in _answers:
    for _answer_2 in _answers:
        print(_answer_1['ans_text'], _answer_2['ans_text'], cos_sim(_answer_1['vector'], _answer_2['vector']))

さいたま市は、埼玉県南部にある政令市です。 さいたま市は、埼玉県南部にある政令市です。 1.0000000000000002
さいたま市は、埼玉県南部にある政令市です。 さいたま市は、埼玉県北部にある政令市です。 0.9230769230769232
さいたま市は、埼玉県南部にある政令市です。 さいたま市は、埼玉県南部にある都市です。 0.8770580193070293
さいたま市は、埼玉県北部にある政令市です。 さいたま市は、埼玉県南部にある政令市です。 0.9230769230769232
さいたま市は、埼玉県北部にある政令市です。 さいたま市は、埼玉県北部にある政令市です。 1.0000000000000002
さいたま市は、埼玉県北部にある政令市です。 さいたま市は、埼玉県南部にある都市です。 0.7893522173763263
さいたま市は、埼玉県南部にある都市です。 さいたま市は、埼玉県南部にある政令市です。 0.8770580193070293
さいたま市は、埼玉県南部にある都市です。 さいたま市は、埼玉県北部にある政令市です。 0.7893522173763263
さいたま市は、埼玉県南部にある都市です。 さいたま市は、埼玉県南部にある都市です。 0.9999999999999998
